## Regression


In [ ]:
# Import library

#Load library
import pandas as pd
import numpy as np
import math
import warnings
warnings.filterwarnings("ignore")
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
!pip install xgboost

### 1. Import dan Saving Data

##### a) Append train & test

In [ ]:
df_train = pd.read_csv('train.csv')

In [ ]:
df_train.head()

In [ ]:
df_train.shape

In [ ]:
df_test = pd.read_csv('test.csv')

In [ ]:
df_test.head()

In [ ]:
df_test.shape

In [ ]:
df_sub = pd.read_csv('sample_submission.csv')

In [ ]:
df_sub.head()

In [ ]:
df_sub.shape

In [ ]:
# import pandas_profiling

# train.iloc[:,:10].profile_report()

In [ ]:
df_train.columns

In [ ]:
df_test.columns

In [ ]:
#FUNCTION FOR SELECTING CATEGORICAL AND NUMERICAL VARIABLE
def categorical(df):
    num=list(df.columns[df.dtypes=='object'])
    return num
    
def numerical(df):
    cat=list(df.columns[df.dtypes=='int64'])+list(df.columns[df.dtypes=='float64'])
    return cat

In [ ]:
num = numerical(df_train)
print(num,'\n')
print('lenght of numerical variable :',len(num))

In [ ]:
num_cor=df_train[num].corr()["SalePrice"].sort_values(ascending=False)
num_cor

In [ ]:
#adding the variable 'SalePrice' to test with any number
df_test['SalePrice']=-100

In [ ]:
#append all data
df = df_train.append(df_test)

In [ ]:
df.head()

### 2. Data Preparation & Feature Engineering

##### a) Handling duplicate data

In [ ]:
# checking duplicated
df.duplicated().sum()

In [ ]:
# if there are duplicating data, then you can use this one to drop it
# df_drop = df.drop_duplicates()

##### b) Checking missing value

In [ ]:
df.isnull().sum()

In [ ]:
df.dtypes

In [ ]:
#function for checking missing value
def check_missing(df):
    percent=df.isnull().sum()/df.shape[0]*100
    freq=df.isnull().sum()
    types=df.dtypes
    df_miss=pd.DataFrame({'percentage':percent, 'frequency':freq, 'var_type':types})
    df_miss.sort_values(by='frequency',ascending= False, inplace=True)
    return df_miss[df_miss['percentage']>0]

In [ ]:
df_check=check_missing(df)
df_check.head(15)

##### c) Separating target variable & feature

In [ ]:
df.head()

In [ ]:
y = df['SalePrice']

In [ ]:
X = df
X.drop(['Id','SalePrice'],axis=1,inplace=True)

In [ ]:
X.head()

##### d) Defining numerical & categorical feature

In [ ]:
#FUNCTION FOR SELECTING CATEGORICAL AND NUMERICAL VARIABLE
def categorical(df):
    num=list(df.columns[df.dtypes=='object'])
    return num
    
def numerical(df):
    cat=list(df.columns[df.dtypes=='int64'])+list(df.columns[df.dtypes=='float64'])
    return cat

In [ ]:
X_cat = categorical(X)
print(X_cat,'\n')
print('lenght of categorical variable :',len(X_cat))

In [ ]:
X_num = numerical(X)
print(X_num,'\n')
print('lenght of numerical variable :',len(X_num))

In [ ]:
X[X_cat].describe()

In [ ]:
X[X_num].describe()

In [ ]:
a=X[X_num]
a.loc[234,:]

##### e) Imputation missing value

In [ ]:
#imputation for numerical variable 
for i in X_num:
    X[i].fillna(X[i].mean(),inplace=True)

In [ ]:
X[X_num].loc[234,:]

In [ ]:
#imputation for categorical variable 
for i in X_cat:
    X[i].fillna('no cat',inplace=True)

In [ ]:
check_missing(X)
#imputation has finished

#####  f) Standarization

In [ ]:
X[X_num].head()

In [ ]:
standard = StandardScaler()
standard.fit(X[X_num])

In [ ]:
X_standard = pd.DataFrame(standard.transform(X[X_num]))
X_standard.columns = X[X_num].columns

In [ ]:
X_standard.head()

In [ ]:
Var_cat = X[X_cat]
Var_cat.loc[234,:]

##### g) One Hot Encoder

###### 1st Data set : X_num + X_cat + y

In [ ]:
# df_full= pd.concat([X[X_num], X[X_cat],y], axis=1)

###### 2nd Dataset : X_standard + X_cat + y

In [ ]:
X_standard.reset_index(drop=True, inplace=True)
Var_cat.reset_index(drop=True, inplace=True)
y.reset_index(drop=True, inplace=True)

In [ ]:
df_full= pd.concat([X_standard, Var_cat ,y], axis=1)

###### Checking effect of numerical imputation

In [ ]:
df_full_train=df_full[df_full['SalePrice']>=0]
df_full_test=df_full[df_full['SalePrice']<0]

In [ ]:
num2 = numerical(df_full_train)
print(num2,'\n')
print('lenght of numerical variable :',len(num2))

In [ ]:
# after
num_cor2=df_full_train[num2].corr()["SalePrice"].sort_values(ascending=False)
num_cor2

In [ ]:
# before
num_cor

In [ ]:
# df_full[['KitchenQual','SalePrice']].groupby('KitchenQual').median().sort_values(by='SalePrice',ascending=False).index

In [ ]:
df_full['KitchenQualNum']=df_full['KitchenQual'].map({'Ex':4,'Gd':3,'TA':2,'Fa':1,'no cat':0})

In [ ]:
# df_full[['PoolQC','SalePrice']].groupby('PoolQC').median().sort_values(by='SalePrice',ascending=False).index

In [ ]:
df_full['PoolQCNum']=df_full['PoolQC'].map({'Ex':3,'Fa':2,'Gd':1,'no cat':0})

In [ ]:
df_full[['Neighborhood','SalePrice']].groupby('Neighborhood').median().sort_values(by='SalePrice',ascending=False).index

In [ ]:
len(['NridgHt', 'NoRidge', 'StoneBr', 'Timber', 'Somerst', 'Veenker',
       'Crawfor', 'ClearCr', 'CollgCr', 'Blmngtn', 'NWAmes', 'Gilbert',
       'SawyerW', 'Mitchel', 'NPkVill', 'NAmes', 'SWISU', 'Blueste', 'Sawyer',
       'BrkSide', 'Edwards', 'OldTown', 'BrDale', 'IDOTRR', 'MeadowV'])

In [ ]:
df_full['NeighborhoodNum']=df_full['Neighborhood'].map({'NridgHt':25,'NoRidge':24,'StoneBr':23,'Timber':22,'Somerst':21,
                                             'Veenker':20,'Crawfor':19,'ClearCr':18,'CollgCr':17,'Blmngtn':16,
                                             'NMAmes':15,'Gilbert':14,'SawyerW':13,'Mitchel':12,'NPkVill':11,
                                             'NAmes':10,'SWISU':9,'Blueste':8,'Sawyer':7,'BrkSide':6,'Edwards':5,
                                             'OldTown':4,'Brdale':3,'IDOTRR':2,'MeadowV':1})

In [ ]:
fitur_num=['SalePrice',
 'OverallQual',
 'GrLivArea',
 'GarageCars',
 'GarageArea',
 'TotalBsmtSF',
 '1stFlrSF',
 'FullBath',
 'TotRmsAbvGrd',
 'YearBuilt',
 'YearRemodAdd']

In [ ]:
best_fitur = fitur_num+['KitchenQualNum','PoolQCNum','NeighborhoodNum']
best_fitur

In [ ]:
df_dummies=pd.get_dummies(df_full,drop_first=True)

In [ ]:
df_dummies.columns

In [ ]:
df_dummies_train=df_dummies[df_dummies['SalePrice']>=0]
df_dummies_test=df_dummies[df_dummies['SalePrice']<0]

In [ ]:
y = df_dummies_train['SalePrice']

In [ ]:
df_dummies_train.drop('SalePrice',axis=1,inplace=True)
df_dummies_test.drop('SalePrice',axis=1,inplace=True)

In [ ]:
df_dummies_train.columns[0]

In [ ]:
fitur=list(df_dummies_train.columns)
len(fitur)

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(df_dummies_train[fitur],
                                                    y,random_state=10)

In [ ]:
# from sklearn.base import BaseEstimator, TransformerMixin
# from sklearn.preprocessing import Imputer

# from statsmodels.stats.outliers_influence import variance_inflation_factor

In [ ]:
# # VIF

# from statsmodels.stats.outliers_influence import variance_inflation_factor

# class ReduceVIF(BaseEstimator, TransformerMixin):
#     def __init__(self, thresh=5.0, impute=True, impute_strategy='median'):
#         # From looking at documentation, values between 5 and 10 are "okay".
#         # Above 10 is too high and so should be removed.
#         self.thresh = thresh
        
#         # The statsmodel function will fail with NaN values, as such we have to impute them.
#         # By default we impute using the median value.
#         # This imputation could be taken out and added as part of an sklearn Pipeline.
#         if impute:
#             self.imputer = Imputer(strategy=impute_strategy)

#     def fit(self, X, y=None):
#         print('ReduceVIF fit')
#         if hasattr(self, 'imputer'):
#             self.imputer.fit(X)
#         return self

#     def transform(self, X, y=None):
#         print('ReduceVIF transform')
#         columns = X.columns.tolist()
#         if hasattr(self, 'imputer'):
#             X = pd.DataFrame(self.imputer.transform(X), columns=columns)
#         return ReduceVIF.calculate_vif(X, self.thresh)

#     @staticmethod
#     def calculate_vif(X, thresh=5.0):
#         # Taken from https://stats.stackexchange.com/a/253620/53565 and modified
#         dropped=True
#         while dropped:
#             variables = X.columns
#             dropped = False
#             vif = [variance_inflation_factor(X[variables].values, X.columns.get_loc(var)) for var in X.columns]
            
#             max_vif = max(vif)
#             if max_vif > thresh:
#                 maxloc = vif.index(max_vif)
#                 print(f'Dropping {X.columns[maxloc]} with vif={max_vif}')
#                 X = X.drop([X.columns.tolist()[maxloc]], axis=1)
#                 dropped=True
#         return X

In [ ]:
# X_num[0:5]

In [ ]:
# transformer = ReduceVIF()

# # Only use 10 columns for speed in this example
# VIF_var = transformer.fit_transform(x_train[X_num], y_train)

# VIF_var.head()

### 3.Modelling

##### 3.1 Train Model

In [ ]:
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

kf5=KFold(n_splits=5,random_state=10)

def eval(alg,kf,x_train, x_test):
    alg.fit(x_train,y_train)
    print("R2 test: \n",alg.score(x_test,y_test),'\n')
    print("R2 train: \n",alg.score(x_train,y_train),'\n')
    
    cvr=cross_val_score(alg,x_train,y_train,cv=kf)
    print("R2 cv : \n",cvr,'\n')
    print("R2 cv mean : \n",cvr.mean(),'\n')
    print("R2 cv std : \n",cvr.std(),'\n')
    

    resi=alg.predict(x_test)-y_test
    plt.scatter(alg.predict(x_test),resi)
    plt.show()

In [ ]:
from xgboost.sklearn import XGBRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.neighbors import KNeighborsRegressor

In [ ]:
rexgb = XGBRegressor(learning_rate=0.1,
                    n_estimators=40, 
                    min_child_weight=3,
                    max_depth=5,
                    gamma=0.1,
                    subample=0.8,
                    colsample_bytree=0.6,
                    reg_alpha=0.1,
                    reg_lambda=0.1,
                    random_state=10,
                    objective='reg:squarederror')

###### LASSO Regression

In [ ]:
###Model Ridge regression
from sklearn.linear_model import Ridge, RidgeCV, ElasticNet, LassoCV, LassoLarsCV
from sklearn.model_selection import cross_val_score

In [ ]:
###Model Lasso regression
lasso = LassoCV()

##### 3.2 Evaluate Model

In [ ]:
# XBG Regressor
eval(rexgb,5,x_train, x_test)

In [ ]:
# LASSO Regression
eval(lasso,5,x_train, x_test)

In [ ]:
### Selected variables
coef = pd.Series(lasso.coef_, index = x_train.columns)
coef

In [ ]:
print("Lasso picked " + str(sum(coef != 0)) + " variables and eliminated the other " +  str(sum(coef == 0)) + " variables")

In [ ]:
import matplotlib

In [ ]:
imp_coef = pd.concat([coef.sort_values().head(10),
                     coef.sort_values().tail(10)])
matplotlib.rcParams['figure.figsize'] = (8.0, 10.0)
imp_coef.plot(kind = "barh")
plt.title("Coefficients in the Lasso Model")

In [ ]:
selected_var = list(coef[coef!=0].index)
selected_var

######  3.3 Choosing best model & feature

###### 3.3 Importance feature of best model

In [ ]:
import seaborn as sns
def feature_importance_df(model,FITUR,k,scale):
    model.fit(x_train,y_train)
    plt.figure(figsize=(6,10))
    sns.set_context("paper", font_scale=scale)
    fitur_imp=pd.DataFrame({'fitur':FITUR,'importances':model.feature_importances_})
    fitur_imp.sort_values(by='importances',ascending=False,inplace=True)
    sns.barplot(x='importances',y='fitur',data=fitur_imp[0:k])
    plt.xlabel("Feature Importances")
    plt.ylabel("Feature")
    plt.show()
    
    return fitur_imp['fitur'][0:k]

In [ ]:
feature_importance = feature_importance_df(rexgb,fitur,30,2)

In [ ]:
feature_importance

In [ ]:
# Modeling with 30 importance variables
eval(rexgb,kf5,x_train[feature_importance], x_test[feature_importance])

In [ ]:
# corrmat = x_train[feature_importance].corr()

# mask = np.zeros_like(corrmat)
# mask[np.triu_indices_from(mask)] = True

# plt.figure(figsize=(6,6))
# sns.heatmap(corrmat,vmax=0.8,mask=mask,square=True)
# sns.set_context('paper', font_scale=2)
# plt.show()

######  3.4 Tunning Parameter

In [ ]:
#load library
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectFromModel

#building function
def tunning_result(alg):
    print('Best Parameter : \n', alg.best_params_,'\n')
    print('Best R-Square  : \n', alg.best_score_,'\n')
    print('Best Model     : \n', alg.best_estimator_,'\n')

###### Tunning 1 : n_estimator

In [ ]:
rexgb_tune = XGBRegressor(learning_rate=0.1,
                    #n_estimators=40, 
                    min_child_weight=3,
                    max_depth=5,
                    gamma=0.1,
                    subample=0.8,
                    colsample_bytree=0.6,
                    reg_alpha=0.1,
                    reg_lambda=0.1,
                    random_state=10,
                    objective='reg:squarederror')

select = SelectFromModel(rexgb_tune,threshold='median')

pipe = Pipeline([('select',select),('clf',rexgb_tune)])

kf5=KFold(n_splits=5,random_state=10)

# parameters
param_xgb = {
    'clf__n_estimators':[20,40,60,80],
}

rexgb_grid = GridSearchCV(pipe,
                          param_grid=param_xgb,
                          cv=kf5)
rexgb_grid.fit(x_train,y_train)

In [ ]:
tunning_result(rexgb_grid)

In [ ]:
rexgb_tune2 = XGBRegressor(learning_rate=0.1,
                    n_estimators=rexgb_grid.best_params_['clf__n_estimators'], 
                    #min_child_weight=3,
                    #max_depth=5,
                    gamma=0.1,
                    subample=0.8,
                    colsample_bytree=0.6,
                    reg_alpha=0.1,
                    reg_lambda=0.1,
                    random_state=10,
                    objective='reg:squarederror')

select = SelectFromModel(rexgb_tune2,threshold='median')

pipe2 = Pipeline([('select',select),('clf',rexgb_tune2)])

kf5=KFold(n_splits=5,random_state=10)

# parameters
param_xgb2 = {
    'clf__max_depth':range(3,10,2),
    'clf__min_child_weight':range(1,8,2)
}

rexgb_grid2 = GridSearchCV(pipe2,
                          param_grid=param_xgb2,
                          cv=kf5)

rexgb_grid2.fit(x_train,y_train)

###### Tunning 3 : Gamma

In [ ]:
rexgb_tune3 = XGBRegressor(learning_rate=0.25,
                    n_estimators=rexgb_grid.best_params_['clf__n_estimators'], 
                    min_child_weight=rexgb_grid2.best_params_['clf__min_child_weight'],
                    max_depth=rexgb_grid2.best_params_['clf__max_depth'],
                    #gamma=0.1,
                    subsample=0.9,
                    colsample_bytree=0.6,
                    reg_alpha=0.1,
                    reg_lambda=0.1,
                    random_state=10,
                    objective='reg:squarederror')

select = SelectFromModel(rexgb_tune3,threshold='median')

pipe3 = Pipeline([('select',select),('clf',rexgb_tune3)])

kf5=KFold(n_splits=5,random_state=10)

# parameters
param_xgb3 = {
    'clf__gamma':[0,0.01,0.1,0.3,0.5,1]}

rexgb_grid3 = GridSearchCV(pipe3,
                          param_grid=param_xgb3,
                          cv=kf5)

rexgb_grid3.fit(x_train,y_train)

In [ ]:
tunning_result(rexgb_grid3)

###### Tuning 4 : Subsample and Colsample

In [ ]:
rexgb_tune4 = XGBRegressor(learning_rate=0.25,
                    n_estimators=rexgb_grid.best_params_['clf__n_estimators'], 
                    min_child_weight=rexgb_grid2.best_params_['clf__min_child_weight'],
                    max_depth=rexgb_grid2.best_params_['clf__max_depth'],
                    gamma=rexgb_grid3.best_params_['clf__gamma'],
                    #subsample=0.9,
                    #colsample_bytree=0.6,
                    reg_alpha=0.1,
                    reg_lambda=0.1,
                    random_state=10,
                    objective='reg:squarederror')

select = SelectFromModel(rexgb_tune4,threshold='median')

pipe4 = Pipeline([('select',select),('clf',rexgb_tune4)])

kf5=KFold(n_splits=5,random_state=10)

# parameters
param_xgb4 = {
    'clf__colsample_bytree':[0.5,0.6,0.7,0.8,0.9,1],
    'clf__subsample':[0.5,0.6,0.7,0.8,0.9,1]
}

rexgb_grid4 = GridSearchCV(pipe4,
                          param_grid=param_xgb4,
                          cv=kf5)

rexgb_grid4.fit(x_train,y_train)

In [ ]:
tunning_result(rexgb_grid4)

###### Tuning 5 : Regularization Parameter

In [ ]:
rexgb_tune5 = XGBRegressor(learning_rate=0.25,
                    n_estimators=rexgb_grid.best_params_['clf__n_estimators'], 
                    min_child_weight=rexgb_grid2.best_params_['clf__min_child_weight'],
                    max_depth=rexgb_grid2.best_params_['clf__max_depth'],
                    gamma=rexgb_grid3.best_params_['clf__gamma'],
                    subsample=rexgb_grid4.best_params_['clf__subsample'],
                    colsample_bytree=rexgb_grid4.best_params_['clf__colsample_bytree'],
                    #reg_alpha=0.1,
                    #reg_lambda=0.1,
                    random_state=10,
                    objective='reg:squarederror')

select = SelectFromModel(rexgb_tune5,threshold='median')

pipe5 = Pipeline([('select',select),('clf',rexgb_tune5)])

kf5=KFold(n_splits=5,random_state=10)

# parameters
param_xgb5 = {
    'clf__reg_alpha':[0.001,0.01,0.1,1,10,100],
    'clf__reg_lambda':[0.001,0.01,0.1,1,10,100]
}

rexgb_grid5 = GridSearchCV(pipe5,
                          param_grid=param_xgb5,
                          cv=kf5)

rexgb_grid5.fit(x_train,y_train)

In [ ]:
tunning_result(rexgb_grid5)

###### Tunning 6 : Learning rate , n_estimator

In [ ]:
rexgb_tune6 = XGBRegressor(#learning_rate=0.25,
                    n_estimators=rexgb_grid.best_params_['clf__n_estimators'], 
                    min_child_weight=rexgb_grid2.best_params_['clf__min_child_weight'],
                    max_depth=rexgb_grid2.best_params_['clf__max_depth'],
                    gamma=rexgb_grid3.best_params_['clf__gamma'],
                    subsample=rexgb_grid4.best_params_['clf__subsample'],
                    colsample_bytree=rexgb_grid4.best_params_['clf__colsample_bytree'],
                    reg_alpha=rexgb_grid5.best_params_['clf__reg_alpha'],
                    reg_lambda=rexgb_grid5.best_params_['clf__reg_lambda'],
                    random_state=10,
                    objective='reg:squarederror')

select = SelectFromModel(rexgb_tune6,threshold='median')

pipe6 = Pipeline([('select',select),('clf',rexgb_tune6)])

kf5=KFold(n_splits=5,random_state=10)

# parameters

param_xgb6 = {
    'clf__learning_rate':[0.25,0.12,0.1,0.05,0.01]}

rexgb_grid6 = GridSearchCV(pipe6,
                          param_grid=param_xgb6,
                          cv=kf5)

rexgb_grid6.fit(x_train,y_train)

In [ ]:
?GridSearchCV

In [ ]:
rexgb_grid6.

In [ ]:
tunning_result(rexgb_grid6)

##### 3.5 Final Model

In [ ]:
rexgb_final = XGBRegressor(learning_rate=0.1,
                    n_estimators=100, 
                    min_child_weight=1,
                    max_depth=3,
                    gamma=0.1,
                    subample=0.9,
                    colsample_bytree=0.6,
                    reg_alpha=0.01,
                    reg_lambda=10,
                    random_state=10,
                    objective='reg:squarederror')

In [ ]:
eval(rexgb_final,kf5)

###### Our First Model

In [ ]:
eval(rexgb,kf5)

In [ ]:
We can see that R-Square CV has increased from 0.867 to 0.876. That is the best model

Lets Prepare to predict the Submission Data

In [ ]:
# predict the response
y_pred=rexgb_final.predict(df_dummies_test[fitur])

In [ ]:
y_pred2=rexgb_grid6.best_estimator_.predict(df_dummies_test[fitur])

In [ ]:
y_pred2=rexgb_grid6.best_estimator_.predict(df_dummies_test[fitur])

In [ ]:
submission = pd.DataFrame({'Id':df_dummies_test['Id'],'SalePrice':y_pred},
                          columns=['Id','SalePrice']) 

In [ ]:
submission.to_csv('submission_1.csv',index=False)

#### EDA

###### i want to know some condition of the data. i want to know about MISSING VALUES, DATA TYPES, LIST OF VARIABLES. I also want to know about the unlabelled data since it needed to be predicted.

In [2]:
#FUNCTION FOR SELECTING CATEGORICAL AND NUMERICAL VARIABLE
def categorical(df):
    num=list(df.columns[df.dtypes=='object'])
    return num
    
def numerical(df):
    cat=list(df.columns[df.dtypes=='int64'])+list(df.columns[df.dtypes=='float64'])
    return ca

In [3]:
cat = categorical(df)
print(cat,'\n')
print('lenght of categorical variable :',len(cat))

NameError: name 'df' is not defined